In [14]:
import pandas as pd
import requests
import json
from html.parser import HTMLParser
import unicode

In [238]:
parser = HTMLParser()

In [158]:
df = pd.read_csv('udemy_links_07_07_2020.csv')
df.head()

,id,url,title,headlines,objective_summary,level,image,instructors,publish_date,last_update_date
0,950390,https://udemy.com/course/machinelearning/,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,"['Master Machine Learning on Python & R', 'Hav...",All Levels,https://img-a.udemycdn.com/course/480x270/9503...,"['Kirill Eremenko', 'Hadelin de Ponteves', 'Su...",2016-09-13,2020-06-17
1,903744,https://udemy.com/course/python-for-data-scien...,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",['Use Python for Data Science and Machine Lear...,All Levels,https://img-a.udemycdn.com/course/480x270/9037...,['Jose Portilla'],2016-09-18,2020-05-03
2,1754098,https://udemy.com/course/the-data-science-cour...,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",['The course provides the entire toolbox you n...,All Levels,https://img-a.udemycdn.com/course/480x270/1754...,"['365 Careers', '365 Careers Team']",2018-06-21,2020-06-24
3,765242,https://udemy.com/course/r-programming/,R Programming A-Z™: R For Data Science With Re...,Learn Programming In R And R Studio. Data Anal...,"['Learn to program in R at a good level', 'Lea...",All Levels,https://img-a.udemycdn.com/course/480x270/7652...,"['Kirill Eremenko', 'SuperDataScience Team']",2016-02-23,2020-07-06
4,1151632,https://udemy.com/course/deeplearning/,Deep Learning A-Z™: Hands-On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,['Understand the intuition behind Artificial N...,All Levels,https://img-a.udemycdn.com/course/480x270/1151...,"['Kirill Eremenko', 'Hadelin de Ponteves', 'Su...",2017-03-28,2020-06-17


In [127]:
import requests
import json
url = "https://www.udemy.com/api-2.0/course-landing-components/950390/me/?components=curriculum"
response = requests.get(url)
response = json.loads(response.text)

In [159]:
def content_extraction(row):
    section_content = []
    section_name = []
    url = "https://www.udemy.com/api-2.0/course-landing-components/" + str(row['id']) + "/me/?components=curriculum"
    response = requests.get(url)
    response = json.loads(response.text)
    for i in response['curriculum']['data']['sections']:
        section_name.append(i['title'].encode('ascii','ignore'))
        section_elements = []
        for j in i['items']:
            section_elements.append(j['title'].encode('ascii','ignore'))
        section_content.append(section_elements)
    return section_name, section_content

In [160]:
description_df = df.apply(content_extraction, axis=1, result_type='expand')

In [167]:
description_df.rename(columns={0:'section_name',
                       1: 'section_content'},
                      inplace=True)

In [189]:
final = pd.merge(df, description_df, how='inner', left_index=True, right_index=True)

In [260]:
from html.parser import HTMLParser


class myhtmlparser(HTMLParser):
    def __init__(self):
        self.reset()
        self.NEWTAGS = []
        self.NEWATTRS = []
        self.HTMLDATA = []
    def handle_starttag(self, tag, attrs):
        self.NEWTAGS.append(tag)
        self.NEWATTRS.append(attrs)
    def handle_data(self, data):
        self.HTMLDATA.append(data)
    def clean(self):
        self.NEWTAGS = []
        self.NEWATTRS = []
        self.HTMLDATA = []
        
parser = myhtmlparser()
parser.feed(response['description']['data']['description'])

In [312]:
def description_extractor(row):
    url = "https://www.udemy.com/api-2.0/course-landing-components/" + str(row['id']) + "/me/?components=description"
    response = requests.get(url)
    response = json.loads(response.text)
    target_audience = ' '.join(response['description']['data']['target_audiences']).encode('ascii', 'ignore')
    parser.__init__()
    parser.feed(response['description']['data']['description'])
    description = ' '.join(parser.HTMLDATA).encode('ascii', 'ignore')
    return description, target_audience

In [321]:
des_df = df.apply(description_extractor, axis=1, result_type='expand')

In [447]:
des_df.rename(columns={0:'description',
                       1: 'target_audience'},
              inplace=True)
des_df

,description,target_audience
0,Interested in the field of Machine Learning? T...,Anyone interested in Machine Learning. Student...
1,Are you ready to start your path to becoming a...,This course is meant for people with at least ...
2,The Problem Data scientist is one of the best ...,You should take this course if you want to bec...
3,Learn R Programming by doing! There are lots o...,This course is for you if you want to learn ho...
4,***As seen on Kickstarter *** Artificial intel...,Anyone interested in Deep Learning Students wh...
5,Extremely Hands-On... Incredibly Practical... ...,Anybody with an interest in Data Science Anybo...
6,New! Updated for Winter 2019 with extra conte...,Software developers or programmers who want to...
7,Learn Python Programming by doing! \n\n There ...,This course if for you if you want to learn ho...
8,Is statistics a driving force in the industry ...,People who want a career in Data Science Peopl...
9,\n\n ***ASSEENONKICKSTARTER*** Learn key AI co...,"Anyone interested in Artificial Intelligence, ..."


In [455]:
DF = pd.merge(final, des_df, how='inner', left_index=True, right_index=True)

In [456]:
def clean_section_name(text):
    cleaned_text = []
    for i in text:
        cleaned = re.sub('--+', '', i)
        cleaned = re.sub('\(.*\)', '', cleaned)
        cleaned_text.append(cleaned)
    return cleaned_text

In [457]:
DF['section_name'] = DF['section_name'].apply(clean_section_name)

In [458]:
def clean_section_content(text):
    cleaned_total_content = []
    for i in text:
        cleaned_content = []
        for j in i:
            cleaned = re.sub('\(.*\)', '', j) #remove the brackets and anything within it.
            cleaned = re.sub('-?.?Step.?\d?.?-?\)?', '', cleaned) #remove 'STEP 1' and so on from the string.
            cleaned = re.sub('--+', '', cleaned) #remove the '----' part from the string.
            cleaned = re.sub('Part.+/d', '', cleaned)
            cleaned = re.sub('-?.?BONUS', '', cleaned)
            cleaned_content.append(cleaned)
        cleaned = []
        [cleaned.append(i) for i in cleaned_content if i not in cleaned]
        cleaned_total_content.append(cleaned)
    return cleaned_total_content

In [459]:
DF['section_content'] = DF['section_content'].apply(clean_section_content)

In [472]:
def clean_description(text):
    cleaned = re.sub('\n+', '', text)
    return cleaned
DF['description'] = DF['description'].apply(clean_description)

In [486]:
DF.reset_index(drop=True)

,id,url,title,headlines,objective_summary,level,image,instructors,publish_date,last_update_date,section_name,section_content,description,target_audience
0,950390,https://udemy.com/course/machinelearning/,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,"['Master Machine Learning on Python & R', 'Hav...",All Levels,https://img-a.udemycdn.com/course/480x270/9503...,"['Kirill Eremenko', 'Hadelin de Ponteves', 'Su...",2016-09-13,2020-06-17,"[Welcome to the course!, Part 1: Data Preproc...","[[Applications of Machine Learning, : Learning...",Interested in the field of Machine Learning? T...,Anyone interested in Machine Learning. Student...
1,903744,https://udemy.com/course/python-for-data-scien...,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",['Use Python for Data Science and Machine Lear...,All Levels,https://img-a.udemycdn.com/course/480x270/9037...,['Jose Portilla'],2016-09-18,2020-05-03,"[Course Introduction, Environment Set-Up, Jupy...","[[Introduction to the Course, Course Help and ...",Are you ready to start your path to becoming a...,This course is meant for people with at least ...
2,1754098,https://udemy.com/course/the-data-science-cour...,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",['The course provides the entire toolbox you n...,All Levels,https://img-a.udemycdn.com/course/480x270/1754...,"['365 Careers', '365 Careers Team']",2018-06-21,2020-06-24,"[Part 1: Introduction, The Field of Data Scien...",[[A Practical Example: What You Will Learn in ...,The Problem Data scientist is one of the best ...,You should take this course if you want to bec...
3,765242,https://udemy.com/course/r-programming/,R Programming A-Z™: R For Data Science With Re...,Learn Programming In R And R Studio. Data Anal...,"['Learn to program in R at a good level', 'Lea...",All Levels,https://img-a.udemycdn.com/course/480x270/7652...,"['Kirill Eremenko', 'SuperDataScience Team']",2016-02-23,2020-07-06,"[Hit The Ground Running, Core Programming Prin...","[[Welcome to the R Programming Course!, Update...",Learn R Programming by doing! There are lots o...,This course is for you if you want to learn ho...
4,1151632,https://udemy.com/course/deeplearning/,Deep Learning A-Z™: Hands-On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,['Understand the intuition behind Artificial N...,All Levels,https://img-a.udemycdn.com/course/480x270/1151...,"['Kirill Eremenko', 'Hadelin de Ponteves', 'Su...",2017-03-28,2020-06-17,"[Welcome to the course, Part 1 - Artificial N...","[[What is Deep Learning?, Updates on Udemy Rev...",***As seen on Kickstarter *** Artificial intel...,Anyone interested in Deep Learning Students wh...
5,513244,https://udemy.com/course/datascience/,Data Science A-Z™: Real-Life Data Science Exer...,Learn Data Science step by step through real A...,['Successfully perform all steps in a complex ...,All Levels,https://img-a.udemycdn.com/course/480x270/5132...,"['Kirill Eremenko', 'SuperDataScience Team']",2015-08-18,2020-07-06,"[Get Excited, What is Data Science?, Part 1: ...","[[Welcome to Data Science A-Z, : Learning Path...",Extremely Hands-On... Incredibly Practical... ...,Anybody with an interest in Data Science Anybo...
6,671576,https://udemy.com/course/data-science-and-mach...,"Machine Learning, Data Science and Deep Learni...",Complete hands-on machine learning tutorial wi...,['Build artificial neural networks with Tensor...,Beginner Level,https://img-a.udemycdn.com/course/480x270/6715...,"['Sundog Education by Frank Kane', 'Frank Kane']",2016-02-02,2020-07-05,"[Getting Started, Statistics and Probability R...","[[Introduction, Udemy 101: Getting the Most Fr...",New! Updated for Winter 2019 with extra conte...,Software developers or programmers who want to...
7,913448,https://udemy.com/course/python-coding/,Python A-Z™: Python For Data Science With Real...,Pr

In [489]:
DF.to_csv('cleaned_udemy_scraped_final.csv')

In [487]:
DF

,id,url,title,headlines,objective_summary,level,image,instructors,publish_date,last_update_date,section_name,section_content,description,target_audience
0,950390,https://udemy.com/course/machinelearning/,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,"['Master Machine Learning on Python & R', 'Hav...",All Levels,https://img-a.udemycdn.com/course/480x270/9503...,"['Kirill Eremenko', 'Hadelin de Ponteves', 'Su...",2016-09-13,2020-06-17,"[Welcome to the course!, Part 1: Data Preproc...","[[Applications of Machine Learning, : Learning...",Interested in the field of Machine Learning? T...,Anyone interested in Machine Learning. Student...
1,903744,https://udemy.com/course/python-for-data-scien...,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",['Use Python for Data Science and Machine Lear...,All Levels,https://img-a.udemycdn.com/course/480x270/9037...,['Jose Portilla'],2016-09-18,2020-05-03,"[Course Introduction, Environment Set-Up, Jupy...","[[Introduction to the Course, Course Help and ...",Are you ready to start your path to becoming a...,This course is meant for people with at least ...
2,1754098,https://udemy.com/course/the-data-science-cour...,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",['The course provides the entire toolbox you n...,All Levels,https://img-a.udemycdn.com/course/480x270/1754...,"['365 Careers', '365 Careers Team']",2018-06-21,2020-06-24,"[Part 1: Introduction, The Field of Data Scien...",[[A Practical Example: What You Will Learn in ...,The Problem Data scientist is one of the best ...,You should take this course if you want to bec...
3,765242,https://udemy.com/course/r-programming/,R Programming A-Z™: R For Data Science With Re...,Learn Programming In R And R Studio. Data Anal...,"['Learn to program in R at a good level', 'Lea...",All Levels,https://img-a.udemycdn.com/course/480x270/7652...,"['Kirill Eremenko', 'SuperDataScience Team']",2016-02-23,2020-07-06,"[Hit The Ground Running, Core Programming Prin...","[[Welcome to the R Programming Course!, Update...",Learn R Programming by doing! There are lots o...,This course is for you if you want to learn ho...
4,1151632,https://udemy.com/course/deeplearning/,Deep Learning A-Z™: Hands-On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,['Understand the intuition behind Artificial N...,All Levels,https://img-a.udemycdn.com/course/480x270/1151...,"['Kirill Eremenko', 'Hadelin de Ponteves', 'Su...",2017-03-28,2020-06-17,"[Welcome to the course, Part 1 - Artificial N...","[[What is Deep Learning?, Updates on Udemy Rev...",***As seen on Kickstarter *** Artificial intel...,Anyone interested in Deep Learning Students wh...
5,513244,https://udemy.com/course/datascience/,Data Science A-Z™: Real-Life Data Science Exer...,Learn Data Science step by step through real A...,['Successfully perform all steps in a complex ...,All Levels,https://img-a.udemycdn.com/course/480x270/5132...,"['Kirill Eremenko', 'SuperDataScience Team']",2015-08-18,2020-07-06,"[Get Excited, What is Data Science?, Part 1: ...","[[Welcome to Data Science A-Z, : Learning Path...",Extremely Hands-On... Incredibly Practical... ...,Anybody with an interest in Data Science Anybo...
6,671576,https://udemy.com/course/data-science-and-mach...,"Machine Learning, Data Science and Deep Learni...",Complete hands-on machine learning tutorial wi...,['Build artificial neural networks with Tensor...,Beginner Level,https://img-a.udemycdn.com/course/480x270/6715...,"['Sundog Education by Frank Kane', 'Frank Kane']",2016-02-02,2020-07-05,"[Getting Started, Statistics and Probability R...","[[Introduction, Udemy 101: Getting the Most Fr...",New! Updated for Winter 2019 with extra conte...,Software developers or programmers who want to...
7,913448,https://udemy.com/course/python-coding/,Python A-Z™: Python For Data Science With Real...,Pr

In [440]:
for i in trial:
    #print(i)
    #print(re.sub('Step.\d', '', i))
    i = re.sub('-?.?Step.?\d?.?-?\)?', '', i)
    #print(i)
    print(re.sub('Part.\d', '', i))

Simple Linear Regression Intuition 
Simple Linear Regression Intuition 
Make sure you have your Machine Learning A-Z folder ready
Simple Linear Regression in Python 
Simple Linear Regression in Python 
Simple Linear Regression in Python 
Simple Linear Regression in Python 
Simple Linear Regression in Python - BONUS
Simple Linear Regression in R 
Simple Linear Regression in R 
Simple Linear Regression in R 
Simple Linear Regression in R 
Simple Linear Regression
 Convolution Operation
b) - ReLU Layer
 Pooling
 Flattening
 Full Connection
Summary
 azlaa


In [378]:
import re
for i in l:
   # print(i)
    print(re.sub(r'--+', '', i))

Welcome to the course!
 Part 1: Data Preprocessing 
Data Preprocessing in Python
Data Preprocessing in R
 Part 2: Regression 


In [379]:
l

['Welcome to the course!',
 '-------------------- Part 1: Data Preprocessing --------------------',
 'Data Preprocessing in Python',
 'Data Preprocessing in R',
 '-------------------- Part 2: Regression --------------------']

In [291]:
import requests
import json
url = "https://www.udemy.com/api-2.0/course-landing-components/950390/me/?components=description"
response = requests.get(url)
response = json.loads(response.text)

In [308]:
parser.__init__()
parser.feed(response['description']['data']['description'])

In [310]:
' '.join(parser.HTMLDATA).encode('ascii', 'ignore')

'Interested in the field of Machine Learning? Then this course is for you! This course has been designed by two professional Data Scientists so that we can share our knowledge and help you learn complex theory, algorithms, and coding libraries in a simple way. We will walk you step-by-step into the World of Machine Learning. With every tutorial, you will develop new skills and improve your understanding of this challenging yet lucrative sub-field of Data Science. This course is fun and exciting, but at the same time, we dive deep into Machine Learning. It is structured the following way: Part 1 - Data Preprocessing Part 2 - Regression: Simple Linear Regression, Multiple Linear Regression, Polynomial Regression, SVR, Decision Tree Regression, Random Forest Regression Part 3 - Classification: Logistic Regression, K-NN, SVM, Kernel SVM, Naive Bayes, Decision Tree Classification, Random Forest Classification Part 4 - Clustering: K-Means, Hierarchical Clustering Part 5 - Association Rule Le

In [311]:
parser.HTMLDATA

[u'Interested in the field of Machine Learning?',
 u'Then this course is for you!',
 u'This course has been designed by two professional Data Scientists so that we can share our knowledge and help you learn complex theory,',
 u'algorithms, and coding libraries in a simple way.',
 u'We will walk you step-by-step into the World of Machine Learning. With every tutorial, you will develop new skills and improve your understanding of this challenging yet lucrative sub-field of Data Science.',
 u'This course is',
 u'fun and exciting, but at the same time, we dive deep into Machine Learning. It is structured the following way:',
 u'Part 1 - Data Preprocessing',
 u'Part 2 - Regression: Simple Linear Regression, Multiple Linear Regression,',
 u'Polynomial',
 u'Regression,',
 u'SVR, Decision Tree Regression,',
 u'Random Forest Regression',
 u'Part 3 - Classification: Logistic Regression, K-NN, SVM, Kernel SVM, Naive Bayes, Decision Tree Classification,',
 u'Random',
 u'Forest Classification',
 u'